In [ ]:
import polars as pl
import numpy as np
import networkx as nx
import random
import rtsvg
rt = rtsvg.RACETrack()
circles     = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6)]
svg_hdr     = '<svg x="0" y="0" width="768" height="768" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white" />'
svg_ftr     = '</svg>'
svg_circles = []
for i in range(len(circles)):
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.4"/>')
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="0.5" fill="{rt.co_mgr.getColor(i)}"/>')

#rt.tile([svg_hdr + ''.join(svg_circles) + svg_ftr])

In [ ]:
#
# Algorithm source & description from:
# "A simple algorithm for 2D Voronoi diagrams"
# https://www.youtube.com/watch?v=I6Fen2Ac-1U
# https://gist.github.com/isedgar/d445248c9ff6c61cef44fc275cb2398f
#
def isedgarVoronoi(self, S, Box=None, pad=10, use_circle_radius=False, merge_threshold=0.1):
    # return bisector of points p0 and p1 -- bisector is ((x,y),(u,v)) where u,v is the vector of the bisector
    def bisector(p0, p1):
        x, y     = (p0[0] + p1[0])/2.0, (p0[1] + p1[1])/2.0
        uv       = self.unitVector((p0, p1))
        pdx, pdy = uv[1], -uv[0]
        return ((x,y), (pdx,pdy))
    # For the circle version
    def bisectorForCircles(c0, c1):
        uv              = self.unitVector((c0, c1))
        x0, y0          = c0[0] + uv[0] * c0[2], c0[1] + uv[1] * c0[2]
        x1, y1          = c1[0] - uv[0] * c1[2], c1[1] - uv[1] * c1[2]
        x,  y           = (x0+x1)/2.0, (y0+y1)/2.0
        pdx, pdy        = uv[1], -uv[0]
        return ((x,y), (pdx,pdy))
    # returns vertices that intersect the polygon
    def intersects(bisects, poly):
        inters, already_added = [], set()
        for i in range(0, len(poly)):
            p0, p1 = poly[i], poly[(i+1)%len(poly)]
            xy = self.lineSegmentIntersectionPoint((bisects[0], (bisects[0][0] + bisects[1][0], bisects[0][1] + bisects[1][1])),(p0, p1))
            if xy is not None:
                xy = (round(xy[0], 1), round(xy[1], 1))
                too_close = False
                for pt in already_added:
                    if self.segmentLength((pt, xy)) < merge_threshold:
                        too_close = True
                        break
                if too_close == False:
                    inters.append((xy, i, (i+1)%len(poly)))
                    already_added.add(xy)
        return inters
    # conctenate a point, a list, and a point into a new list
    def createCell(my_cell, p0, p1, i0, i1, sign):
        l = [p0]
        i = i0
        while i != i1:
            l.append(my_cell[i])
            i += sign
            if i < 0:             i += len(my_cell)
            if i >= len(my_cell): i -= len(my_cell)
        l.append(my_cell[i1])
        l.append(p1)
        return l
    # contain true if pt is in poly
    def contains(poly, pt):
        inter_count = 0
        for i in range(len(poly)):
            p0, p1 = poly[i], poly[(i+1)%len(poly)]
            _tuple_ = self.segmentsIntersect((pt,(pt[0]+1e9,pt[1])),(p0,p1)) # use a ray from the pt to test
            if _tuple_[0] and (_tuple_[1], _tuple_[2]) != p1: inter_count += 1
        if inter_count%2 == 1: return True
        return False
    # remove any near duplicate points from the polygon
    def removeDuplicatePoints(_poly_):
        _new_ = []
        _new_.append(_poly_[0])
        for i in range(1,len(_poly_)):
            if self.segmentLength((_poly_[i],_new_[-1])) > merge_threshold: _new_.append(_poly_[i])
        return _new_
    # normalize the segment
    def normalizeSegment(segment):
        _xy0_, _xy1_ = segment
        if _xy0_[0] <  _xy1_[0] or (_xy0_[0] == _xy1_[0] and _xy0_[1] < _xy1_[1]): return (_xy0_, _xy1_)
        else:                                                                      return (_xy1_, _xy0_)
    # clumsy ... but this should work / if not very elegantly...
    def accountForChanges(_old_, _new_, _p_, s2p):
        segs_in_old, segs_in_new, xy_to_seg_in_old, xy_to_seg_in_new = set(), set(), {}, {}
        for i in range(len(_old_)):
            _segment_ = normalizeSegment((_old_[i], _old_[(i+1)%len(_old_)]))
            segs_in_old.add(_segment_)
            if _segment_[0] not in xy_to_seg_in_old: xy_to_seg_in_old[_segment_[0]] = set()
            if _segment_[1] not in xy_to_seg_in_old: xy_to_seg_in_old[_segment_[1]] = set()
            xy_to_seg_in_old[_segment_[0]].add(_segment_)
            xy_to_seg_in_old[_segment_[1]].add(_segment_)
        for i in range(len(_new_)):
            _segment_ = normalizeSegment((_new_[i], _new_[(i+1)%len(_new_)]))
            segs_in_new.add(_segment_)
            if _segment_[0] not in xy_to_seg_in_new: xy_to_seg_in_new[_segment_[0]] = set()
            if _segment_[1] not in xy_to_seg_in_new: xy_to_seg_in_new[_segment_[1]] = set()
            xy_to_seg_in_new[_segment_[0]].add(_segment_)
            xy_to_seg_in_new[_segment_[1]].add(_segment_)
        segs_only_in_old = segs_in_old - segs_in_new
        segs_only_in_new = segs_in_new - segs_in_old
        # new should be three -- two should be shortened (and share a single xy with the old set), one should be new (both xy's unique)
        if len(segs_only_in_new) != 3: raise ValueError('segs_only_in_new != 3')
        seg0, seg1, seg2 = list(segs_only_in_new)
        if   seg0[0] not in xy_to_seg_in_old and seg0[1] not in xy_to_seg_in_old: # seg0 is new
            seg_new, seg_single0, seg_single1 = seg0, seg1, seg2
        elif seg1[0] not in xy_to_seg_in_old and seg1[1] not in xy_to_seg_in_old: # seg1 is new
            seg_new, seg_single0, seg_single1 = seg1, seg0, seg2
        elif seg2[0] not in xy_to_seg_in_old and seg2[1] not in xy_to_seg_in_old: # seg2 is new
            seg_new, seg_single0, seg_single1 = seg2, seg0, seg1
        else: raise ValueError('no match [new segment]')
        # transfer points to both seg_single0 and seg_single1
        # ... find the two possible transfers for seg_single0
        if   seg_single0[0] in xy_to_seg_in_old:
            if seg_single0[1] in xy_to_seg_in_old: raise ValueError('seg_single0[1] in xy_to_seg_in_old (and should not be)')
            single0_old_seg0, single0_old_seg1 = list(xy_to_seg_in_old[seg_single0[0]])
        elif seg_single0[1] in xy_to_seg_in_old:
            if seg_single0[0] in xy_to_seg_in_old: raise ValueError('seg_single0[0] in xy_to_seg_in_old (and should not be)')
            single0_old_seg0, single0_old_seg1 = list(xy_to_seg_in_old[seg_single0[1]])
        else: raise ValueError('no match [seg_single0]')

        # ... find the two possible transfers for seg_single1
        if   seg_single1[0] in xy_to_seg_in_old:
            if seg_single1[1] in xy_to_seg_in_old: raise ValueError('seg_single1[1] in xy_to_seg_in_old (and should not be)')
            single1_old_seg0, single1_old_seg1 = list(xy_to_seg_in_old[seg_single1[0]])
        elif seg_single1[1] in xy_to_seg_in_old:
            if seg_single1[0] in xy_to_seg_in_old: raise ValueError('seg_single1[0] in xy_to_seg_in_old (and should not be)')
            single1_old_seg0, single1_old_seg1 = list(xy_to_seg_in_old[seg_single1[1]])
        else: raise ValueError('no match [seg_single1]')

        if (single0_old_seg0 == single1_old_seg0 and single0_old_seg1 == single1_old_seg1) or \
           (single0_old_seg0 == single1_old_seg1 and single0_old_seg1 == single1_old_seg0):
            # special case where the two segments are the same -- happens when the new cell is a triangle
            s2p[seg_single0] = s2p[single0_old_seg0]
            s2p[seg_single1] = s2p[single0_old_seg1]
        else:
            if   single0_old_seg0 in segs_in_new: s2p[seg_single0] = s2p[single0_old_seg1]
            elif single0_old_seg1 in segs_in_new: s2p[seg_single0] = s2p[single0_old_seg0]
            else: raise ValueError('at least one single0_old should be in segs_in_new')
            if   single1_old_seg0 in segs_in_new: s2p[seg_single1] = s2p[single1_old_seg1]
            elif single1_old_seg1 in segs_in_new: s2p[seg_single1] = s2p[single1_old_seg0]
            else: raise ValueError('at least one single1_old should be in segs_in_new')

    # actual algorithm
    if Box is None:
        x_l, x_r, y_t, y_b = S[0][0], S[0][0], S[0][1], S[0][1]
        for pt in S: x_l, x_r, y_t, y_b = min(x_l, pt[0]), max(x_r, pt[0]), max(y_t, pt[1]), min(y_b, pt[1])
        Box = [(x_l-pad, y_t+pad), (x_r+pad, y_t+pad), (x_r+pad, y_b-pad), (x_l-pad, y_b-pad)]
    cells, all_segments_to_points = [], {}
    for i in range(len(S)):
        p                  = S[i]
        cell               = Box
        segments_to_points = {}   # do this on a local basis and then group w/ others at the end
        for j in range(len(cell)):
            segment = normalizeSegment((cell[j], cell[(j+1)%len(cell)]))
            if segment not in segments_to_points: segments_to_points[segment] = set()
            segments_to_points[segment].add(p[:2])
        for q in S:
            if p == q: continue
            B            = bisector(p,q) if use_circle_radius == False else bisectorForCircles(p,q)
            B_intersects = intersects(B, cell)
            if len(B_intersects) == 2:
                t1, t2  = B_intersects[0][0], B_intersects[1][0] # these are _xy_ tuples
                segment = normalizeSegment((t1, t2))
                if segment not in segments_to_points: segments_to_points[segment] = set()
                segments_to_points[segment].add(p[:2]), segments_to_points[segment].add(q[:2]) 
                xi, xj  = B_intersects[0][2], B_intersects[1][1]
                newCell = createCell(cell, t1, t2, xi, xj, 1)
                if contains(newCell, p) == False:
                    xi, xj  = B_intersects[0][1], B_intersects[1][2]
                    newCell = createCell(cell, t1, t2, xi, xj, -1)
                accountForChanges(cell, newCell, p[:2], segments_to_points)
                cell = newCell
        cells.append(cell)
        # transfer segments from local to all
        for j in range(len(cell)):
            segment = normalizeSegment((cell[j], cell[(j+1)%len(cell)]))
            if segment not in all_segments_to_points: all_segments_to_points[segment] = set()
            all_segments_to_points[segment].update(segments_to_points[segment])

    return cells, all_segments_to_points

polys, s2p = isedgarVoronoi(rt, circles, [(0,0),(0,128),(128,128),(128,0)], use_circle_radius=True)
svg_polys, xys_seen = [], set()
for i in range(len(polys)):
    poly = polys[i]
    d = [f'M {poly[0][0]} {poly[0][1]} ']
    xys_seen.add(poly[0])
    for j in range(1,len(poly)): 
        d.append(f'L {poly[j][0]} {poly[j][1]} ')
        xys_seen.add(poly[j])
    d.append('Z')
    svg_polys.append(f'<path d="{"".join(d)}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.5"/>')
for _xy_ in xys_seen:
    svg_polys.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{1.0 + random.random()}" fill="none" stroke="#000000" stroke-width="0.1"/>')
svg_s2p = []
for _segment_ in s2p:
    x0,y0,x1,y1 = _segment_[0][0], _segment_[0][1], _segment_[1][0], _segment_[1][1]
    for _xy_ in s2p[_segment_]:
        color_i = 0
        for i in range(len(circles)):
            if _xy_ == circles[i][:2]:
                color_i = i
                break
        svg_s2p.append(f'<line x1="{x0}" y1="{y0}" x2="{_xy_[0]}" y2="{_xy_[1]}" stroke="{rt.co_mgr.getColor(color_i)}" stroke-width="0.5"/>')
        svg_s2p.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{x1}" y2="{y1}" stroke="{rt.co_mgr.getColor(color_i)}" stroke-width="0.5"/>')
for _segment_ in s2p:
    x0,y0,x1,y1 = _segment_[0][0], _segment_[0][1], _segment_[1][0], _segment_[1][1]
    svg_s2p.append(f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="#000000" stroke-width="0.05"/>')

rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_polys) + svg_ftr,
         svg_hdr + ''.join(svg_circles) + ''.join(svg_s2p)   + svg_ftr])